In [1]:
import pandas as pd
import requests
import time
import json as js
import gmaps

from config import gkey

In [2]:
#Open file to get working_events copy to use 
working_set_df = pd.read_csv('events.csv') 
print(working_set_df.columns)
#working_set_df.head()
#now has the index column

Index(['ID', 'Scheduled', 'PassThru', 'Start Time', 'End Time', 'Event',
       'School Name', 'School Code', 'Sport'],
      dtype='object')


In [3]:
Geo_df=pd.DataFrame()
Geo_df["School Name"]=""
Geo_df["Lat"]=""
Geo_df["Long"]=""

#working_set_df["Lat"]=''
#working_set_df["Long"]=''
radii=50000
target_city=""
target_type="university"
keys=""

params={"address":target_city,
        "radius":radii,
        "types":target_type,
        "keyword":keys,
        "key":gkey}
base_url="https://maps.googleapis.com/maps/api/geocode/json"
basecount=working_set_df.count()

for index,row in working_set_df.iterrows():
    target_city=row['School Name']
    #print(target_city)
    params["address"]=target_city
    params["keyword"]=target_city
    response=requests.get(base_url, params=params)
    #print(response)
    new_geo=response.json()
    try:
        Geo_df.loc[index,"School Name"]=row['School Name']
        Geo_df.loc[index,"Lat"]=new_geo["results"][0]["geometry"]["location"]["lat"]
        Geo_df.loc[index,"Long"]=new_geo["results"][0]["geometry"]["location"]["lng"]
        #here are the working_set_df added columns
        #working_set_df["Lat"]=new_geo["results"][0]["geometry"]["location"]["lat"]
        #working_set_df["Long"]=new_geo["results"][0]["geometry"]["location"]["lng"]
    except(KeyError, IndexError):
        print('Skipping')



Skipping
Skipping
Skipping
Skipping


In [4]:
print(basecount)
Geo_df


ID             557
Scheduled      557
PassThru       557
Start Time     557
End Time       557
Event          557
School Name    557
School Code    557
Sport          557
dtype: int64


,School Name,Lat,Long
0,Conference USA,32.9353,-96.984
1,Davenport University,42.8495,-85.5307
2,University of West Georgia,33.5718,-85.1032
3,Presidents Athletic Conference,41.412,-80.3839
4,Conference USA,32.9353,-96.984
...,...,...,...
552,Providence College,41.8439,-71.4349
553,Mississippi State University,33.4552,-88.7944
554,East Carolina University,35.6069,-77.3665
555,University of the Cumberlands,36.7371,-84.1638


In [5]:
Geo_df.to_csv('Geo_df.csv')

In [49]:
#Geo_df = pd.read_csv('Geo_df.csv')
#Geo_df.loc[1:70,:]

Geo_df.columns

Index(['School Name', 'Lat', 'Long'], dtype='object')

In [51]:
# Store latitude and longitude in locations
locations=pd.DataFrame()
for index, row in Geo_df.iterrows():
    if row['Lat']> -90 or row['Lat']<90:
        locations.loc[index,'Lat']=row['Lat']
        locations.loc[index,'Long']=row['Long']
  
      

print(locations.count())

Lat     553
Long    553
dtype: int64


In [52]:
import gmaps
# Customize the size of the figure
figure_layout = {
    'width': '900px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '0px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
#fig

In [53]:
# Assign the marker layer to a variable
markers = gmaps.heatmap_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
#plt.savefig("Home_Team_Heatmap.png")
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='0px', wi…

In [54]:
states_df = []

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/geocode/json?latlng="
key = "&key=" + gkey
    
for index, rows in locations.iterrows():
    try:
        url = f"{base_url}{rows.Lat},{rows.Long}{key}"
        #print(requests.get(url).json()["results"][0]["address_components"][4]["long_name"])

        states_df.append({
                "Lat":rows.Lat,
                "Long":rows.Long,
                "State":requests.get(url).json()["results"][0]["address_components"][4]["long_name"]
        })  
    except(KeyError, IndexError):
        print('Skipping')

#states_df.head()

Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping
Skipping


In [55]:
states_df = pd.DataFrame(states_df)
states_df
states_count = states_df['State'].value_counts()
states_count 

Texas               54
United States       38
Lehigh County       37
Pennsylvania        30
Kentucky            22
                    ..
Wayne Township       1
Milwaukee County     1
Chapel Hill          1
Howard County        1
Hamilton County      1
Name: State, Length: 83, dtype: int64

Notes:  This uses the events.csv - which was the first data frame I created - it takes the school name and queries google maps api to get the Longitude and Latitude for the school location.  Then the locations can be put into the heat map.  I did not put the locations back into the events data frame but you could.  I put in commented lines to show you how you would do this.  But the reason the location data was set up as it's own data frame was to allow the maps api to use data that had no values that would be detected as null or bad Latitude values.